In [ ]:
import sys
import os

env = "colab" # "colab" or "local"

if env == "colab":
    %cd /content/sample_data/
    !git clone https://github.com/ScisciJP/scisciJP2024_tutorial.git
    path = '/content/sample_data/scisciJP2024_tutorial'
if env == "local":
    path = 'YOUR_PATH'

sys.path.append(path)
print(os.getcwd())

os.chdir(path)
sys.path

In [ ]:
%pip install pyalex 
%pip install japanize_matplotlib 
from pyalex import Works, Authors, Sources, Institutions, Concepts, Funders
import pyalex

import pandas as pd
import numpy as np
import math
import tqdm
import matplotlib.pyplot as plt
import japanize_matplotlib

import util

In [ ]:
pyalex.config.email = "your@email.address" 

In [ ]:
# institutions_subset = util.getAll(Institutions().filter(country_code="JP", works_count=">50").select(["id","display_name","works_count","summary_stats"]))
# pd.to_pickle(pd.DataFrame(institutions_subset), "./supplementary/institutions_subset.pickle")

institutions_subset = pd.read_pickle("./supplementary/institutions_subset.pickle")
topics = pd.read_csv("./supplementary/OpenAlex_topic_mapping_table.csv")
subfields = topics[["subfield_id","subfield_name"]].drop_duplicates().set_index("subfield_id")

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="Boolean Series key will be reindexed to match DataFrame index")

sample_subfields = subfields.sample(15).index
for field_id in sample_subfields:
    pyalexObj = Works().filter(primary_topic={"subfield":{"id":field_id}},
                            institutions_distinct_count=">1", 
                            cited_by_count=">0",
                            authorships={"institutions":{"country_code":"JP"}},
                            from_publication_date="2010-01-01")\
                        .select(["id","authorships","display_name"])
    count = min(5000, pyalexObj.count())
    print("分野：",subfields.loc[field_id,"subfield_name"],"における複数機関の共著論文を取得中")
    w = util.getAll(pyalexObj.sample(count,seed=42))
    for i in range(len(w)):
        w[i]["institutions"] = list(map(lambda a: a["institutions"][0]["id"] if len(a['institutions'])>0 else None, w[i]["authorships"]))
        w[i]["countries"] = list(map(lambda a: a["institutions"][0]["country_code"] if len(a['institutions'])>0 else None, w[i]["authorships"]))
    works = pd.DataFrame(w).explode(column=["institutions","countries"])\
        .dropna(subset=["institutions"])\
        .drop_duplicates(subset=["id","institutions"])

    i11l_works = works[works["countries"].groupby(level=0).nunique()>1]
    i11l_institutions = i11l_works[i11l_works["countries"]=="JP"].value_counts("institutions")
    top_institutions = pd.merge(i11l_institutions, pd.DataFrame(institutions_subset),how="left", left_index=True, right_on="id").set_index("id")
    print(subfields.loc[field_id,"subfield_name"])
    print(top_institutions[["display_name","count"]].rename(columns={"display_name":"機関名", "count":"国際共著"}).head(5))
    print("---------------")